# 5. Финальный пайплайн инференса (Pipeline.from_artifacts)
- Короткая демонстрация того, как собирать пайплайн из артефактов проекта
- Одиночный инференс с Grad-CAM и JSON-отчетом
- Пакетный инференс по папке с агрегированным JSON и опциональными per-file JSON
- Заключение

In [1]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

RAW_DIR = ROOT / 'data' / 'raw'
MODELS_DIR = ROOT / 'models'
REPORTS_DIR = ROOT / 'reports'

import json
import torch
# src
from src.final_pipeline import Pipeline

In [2]:
# Проверка CUDA
if torch.cuda.is_available():
    print('CUDA is available!')
    print('Device name:', torch.cuda.get_device_name(0))
    print(f'CUDA version: {torch.version.cuda}')
else:
    print('CUDA is not available.')

CUDA is available!
Device name: NVIDIA GeForce RTX 3060 Laptop GPU
CUDA version: 12.4


## Одиночный инференс
- Загружаем пайплайн из артефактов: модели, нормировок, лучших параметров
- Обрабатываем все `*.dcm` в указанной папке
- Градиентные карты в `reports/gradcam_infer/<имя_файла>/...`
- Агрегированный JSON в `reports/patient_inference.json`
- При `per_file=True` создаются отдельные JSON на каждый файл в `reports/inference/`


In [3]:
class_names = ['Норма - %', 'Пневмония - %', 'Патология легких - %']
dcm_dir = RAW_DIR / 'author' # здесь находится мой реальный снимок легких, который я сделал несколько дней назад, мне предоставили dcm файл

if dcm_dir.exists():
    pipe = Pipeline.from_artifacts(models_dir=MODELS_DIR, reports_dir=REPORTS_DIR)
    agg_report = pipe.infer_dir_to_json(
        dcm_dir,
        class_names=class_names,
        out_json=REPORTS_DIR / 'patient_inference.json',
        per_file=False,
    )
    print(json.dumps(agg_report, indent=2, ensure_ascii=False))
else:
    print('[skip] Папка с DICOM не найдена:', dcm_dir)

Infer author:   0%|          | 0/1 [00:00<?, ?img/s]

{
  "model_accuracy_percent": 73.019,
  "items": [
    {
      "dicom_file": "alexander.dcm",
      "patient_meta": {
        "age": 0.0,
        "sex": "M",
        "view_position": "Unknown",
        "spacing_x": 0.14111110568,
        "spacing_y": 0.14111110568
      },
      "predicted_class": "Норма - %",
      "probabilities": {
        "Норма - %": 97.413,
        "Пневмония - %": 0.155,
        "Патология легких - %": 2.432
      },
      "gradcam_visualization": "reports\\gradcam_infer\\alexander\\gradcam_pred_0.png"
    }
  ]
}


## Пакетный инференс
- Обрабатываем все `*.dcm` в указанной папке
- Градиентные карты в `reports/gradcam_infer/<имя_файла>/...`
- Агрегированный JSON в `reports/patient_inference.json`
- При `per_file=True` создаются отдельные JSON на каждый файл в `reports/inference/`

In [ ]:
dcm_dir = RAW_DIR / 'stage_2_test_images'

if dcm_dir.exists():
    pipe = Pipeline.from_artifacts(models_dir=MODELS_DIR, reports_dir=REPORTS_DIR)
    agg_report = pipe.infer_dir_to_json(
        dcm_dir,
        class_names=class_names,
        out_json=REPORTS_DIR / 'patient_inference.json',
        per_file=False,
    )
    print(json.dumps(agg_report, indent=2, ensure_ascii=False))
else:
    print('[skip] Папка с DICOM не найдена:', dcm_dir)

Infer stage_2_test_images:   0%|          | 0/3000 [00:00<?, ?img/s]

# Финальное заключение по модели **ResNet50 (Transfer + Optuna + Meta)**

В ходе проекта была разработана и поэтапно улучшена модель глубокого обучения для автоматического анализа рентгеновских снимков грудной клетки с целью выявления пневмонии или других патологий легких.

Исследование включало три ключевых этапа:

1. **Базовая модель ResNet18** - определяла исходный уровень точности и позволила сформировать контрольную точку сравнения.  
2. **ResNet50 (Transfer + Optuna)** - улучшенная архитектура на предобученных весах *ImageNet*, оптимизированная с помощью подбора гиперпараметров
(*Optuna*, 100 trial’ов).
3. **ResNet50 Final (Transfer + Optuna + Meta)** - итоговая версия модели, дополненная мета-информацией о пациенте (возраст, пол, тип проекции, параметры *spacing*), и адаптированная для инференса в продакшен-сценариях.

Добавление мета-признаков обеспечило точечный, устойчивый прирост метрик (особенно *F1-weighted* и *PR-AUC*) и заметное улучшение на патологических классах (*Lung Opacity*, *Not Normal*).
Модель стала чувствительнее к признакам заболевания, при этом сохранив общую устойчивость.

---

## Финальное тестирование и продакшен-пайплайн

В финальной стадии был реализован **инференс-пайплайн (Pipeline)**, который объединяет:
- чтение и обработку DICOM/PNG снимков;
- нормализацию мета-признаков и предсказание модели;
- генерацию визуализаций **Grad-CAM**;
- формирование итогового **JSON-отчета**, содержащего:
  - мета-информацию пациента (возраст, пол, проекция, spacing);
  - вероятности по каждому классу;
  - путь к Grad-CAM визуализации.

Пример фрагмента отчета:
```json
{
  "model_accuracy_percent": 73.019,
  "items": [
    {
      "dicom_file": "alexander.dcm",
      "patient_meta": {
        "age": 0.0,
        "sex": "M",
        "view_position": "Unknown",
        "spacing_x": 0.14111110568,
        "spacing_y": 0.14111110568
      },
      "predicted_class": "Норма - %",
      "probabilities": {
        "Норма - %": 97.413,
        "Пневмония - %": 0.155,
        "Патология легких - %": 2.432
      },
      "gradcam_visualization": "reports\\gradcam_infer\\alexander\\gradcam_pred_0.png"
    }
  ]
}
```

Данная модель имеет общую точность предсказания в **73 процента**. Для использования в исследовательских целях или в качестве дополнительного инструмента врача этого достаточно, но для медицинского продакшена нужно расширение данных и дополнительное дообучение модели, чтобы довести точность модели до 90+ процентов.